In [1]:
import datasource
import torch
import sklearn.preprocessing
import numpy as np
from ray.air import session
from ray.air.checkpoint import Checkpoint


def train_loop_per_worker(config):
    def _import_class(name:str):
        import importlib
        index = name.rfind('.')
        module_name = name[:index] if index != -1 else '__main__'
        class_name = name[index + 1:]
        return getattr(importlib.import_module(module_name), class_name)
    
    model = config['model']
    criterion = config['criterion']
    optimizer = config['optimizer']
    imputer = config['imputer']
    scaler = config['scaler']

    data = datasource.get_data()
    train_indexes, test_indexes = datasource.get_index_splited_by_time(data)

    index_X = 'FSR_for_force'
    index_y = 'force'
    concated_train_indexes = np.concatenate(train_indexes)
    if imputer:
        imputer = _import_class(imputer)(**config['imputer_args'])
        imputer.fit(data.loc[concated_train_indexes, [index_X, index_y]])
    if scaler:
        scaler_X = _import_class(scaler)()
        scaler_y = _import_class(scaler)()
        scaler_X.fit(data.loc[concated_train_indexes, index_X])
        scaler_y.fit(data.loc[concated_train_indexes, index_y])
    data.loc[:, index_X] = scaler_X.transform(data.loc[:, index_X])
    data.loc[:, index_y] = scaler_y.transform(data.loc[:, index_y])
    train_dataset = datasource.FSRDataset(data.loc[:, index_X], data.loc[:, index_y], train_indexes)
    test_dataset = datasource.FSRDataset(data.loc[:, index_X], data.loc[:, index_y], test_indexes)

    model = _import_class(model)(input_size=len(data.loc[:, index_X].columns), output_size=len(data.loc[:, index_y].columns), **config['model_args'])
    criterion = _import_class(criterion)()
    optimizer = _import_class(optimizer)(model.parameters(), **config['optimizer_args'])

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=None)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=None)

    while True:
        model.train()
        for X, y in train_loader:
            pred = model(X)
            loss = criterion(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        model.eval()
        with torch.inference_mode():
            mae, mse, mape, num = [], [], [], []
            for X, y in test_loader:
                pred = model(X)
                if scaler:
                    pred = scaler_y.inverse_transform(pred)
                    y = scaler_y.inverse_transform(y)
                mae.append(sklearn.metrics.mean_absolute_error(y, pred))
                mse.append(sklearn.metrics.mean_squared_error(y, pred))
                mape.append(sklearn.metrics.mean_absolute_percentage_error(y, pred))
                num.append(len(y))
            mae = np.average(mae, weights=num)
            mse = np.average(mse, weights=num)
            mape = np.average(mape, weights=num)
            rmse = mse ** 0.5
        session.report(
            dict(rmse=rmse, mae=mae, mape=mape),
            checkpoint=Checkpoint.from_dict(
                dict(model=model.state_dict(), optimizer=optimizer.state_dict()),
            ),
        ),

In [ ]:
from 

In [2]:
from ray.air.config import ScalingConfig, RunConfig, CheckpointConfig
from ray.air.integrations.wandb import WandbLoggerCallback
from ray.train.torch import TorchTrainer
from ray.tune.stopper import TrialPlateauStopper, ExperimentPlateauStopper, CombinedStopper
from ray.data.preprocessors import Chain, SimpleImputer, MaxAbsScaler, MinMaxScaler, PowerTransformer, RobustScaler, StandardScaler

trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker, 
    train_loop_config={
        'model':'fsr_model.LSTM',
        'model_args':{
            'hidden_size':8,
            'num_layer':1,
        },
        'criterion':'torch.nn.MSELoss',
        'optimizer':'torch.optim.Adam',
        'optimizer_args':{
            'lr': 1e-3,
        },
        'scaler':'sklearn.preprocessing.StandardScaler',
    },
    scaling_config=ScalingConfig(
        num_workers=2,
        use_gpu=False,
    ),
    run_config=RunConfig(
        # callbacks=[
        #     WandbLoggerCallback(project='FSR-prediction'),
        # ],
        stop=CombinedStopper(
            TrialPlateauStopper(metric='rmse'),
            ExperimentPlateauStopper(metric='rmse'),
        ),
        checkpoint_config=CheckpointConfig(
            num_to_keep=3,
            checkpoint_score_attribute='rmse',
            checkpoint_score_order='min',
        ),
    ),
    datasets={
        'train':,
    }
    preprocessor=Chain(
        
    )
)
result = trainer.fit()
print(f"Last result: {result.metrics}")

2023-07-03 21:58:27,833	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-07-03 21:58:30,149	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Trainer(...)`.
(TorchTrainer pid=937528) 2023-07-03 21:58:36,758	INFO backend_executor.py:137 -- Starting distributed worker processes: ['937591 (172.26.215.93)', '937592 (172.26.215.93)']
(RayTrainWorker pid=937591) 2023-07-03 21:58:38,515	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=2]


Trial name,date,done,hostname,iterations_since_restore,mae,mape,node_ip,pid,rmse,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
TorchTrainer_4f0ba_00000,2023-07-03_21-59-13,False,DESKTOP-0P789CI,56,94.1527,1.5377e+08,172.26.215.93,937528,181.656,True,39.1924,0.668801,39.1924,1688389153,56,4f0ba_00000


In [2]:
from glob import glob
import pandas as pd
paths = glob('./data/*/*/*')
filedata = pd.DataFrame([path.split('/')[2:] for path in paths], columns=['subject', 'pose', 'filename'])
filedata['path'] = paths
filedata = filedata.sort_values(['subject', 'pose']).reset_index(drop=True)
filedata
data = []
for index, value in filedata.iterrows():
    df = pd.read_pickle(value['path'])
    df = df.rename_axis('time').reset_index()
    df['id'] = index
    cols = df.columns.to_list()
    df = df[cols[-1:] + cols[:-1]]
    data.append(df)
data = pd.concat(data)
data = data.reset_index(drop=True)
data

Category  id time force                             x_coord      ...  \
Position              A    B     C      D    E    F       A   B  ...   
0          0    0   0.0  0.0   0.0   22.0  0.0  0.0     NaN NaN  ...   
1          0    1   0.0  0.0   0.0  277.0  0.0  0.0     NaN NaN  ...   
2          0    2   0.0  0.0   0.0  488.0  0.0  0.0     NaN NaN  ...   
3          0    3   0.0  0.0   0.0  501.0  0.0  0.0     NaN NaN  ...   
4          0    4   0.0  0.0   0.0  540.0  0.0  0.0     NaN NaN  ...   
...       ..  ...   ...  ...   ...    ...  ...  ...     ...  ..  ...   
52144     72  635   0.0  0.0   0.0  983.0  0.0  0.0     NaN NaN  ...   
52145     72  636   0.0  0.0   0.0  962.0  0.0  0.0     NaN NaN  ...   
52146     72  637   0.0  0.0   0.0  910.0  0.0  0.0     NaN NaN  ...   
52147     72  638   0.0  0.0   0.0  851.0  0.0  0.0     NaN NaN  ...   
52148     72  639   0.0  0.0  10.0  839.0  0.0  0.0     NaN NaN  ...   

Category FSR_for_force                  FSR_for_coord                 \
Position             C      D    E    F             A    B         C   
0                  0.0    0.0  0.0  0.0           0.0  0.0  0.000000   
1                  0.0   60.0  0.0  0.0           0.0  0.0  0.000000   
2                  0.0   73.0  0.0  0.0           0.0  0.0  0.000000   
3                  0.0   84.0  0.0  0.0           0.0  0.0  0.000000   
4                  0.0  100.0  0.0  0.0           0.0  0.0  0.000000   
...                ...    ...  ...  ...           ...  ...       ...   
52144             37.0  180.0  0.0  0.0           0.0  0.0  0.587302   
52145             38.0  172.0  0.0  0.0           0.0  0.0  0.603175   
52146             38.0  165.0  0.0  0.0           0.0  0.0  0.603175   
52147             39.0  160.0  0.0  0.0           0.0  0.0  0.619048   
52148             39.0  156.0  0.0  0.0           0.0  0.0  0.619048   

Category                      
Position         D    E    F  
0         0.000000  0.0  0.0  
1         0.952381  0.0  0.0  
2         1.158730  0.0  0.0  
3         1.333333  0.0  0.0  
4         1.587302  0.0  0.0  
...            ...  ...  ...  
52144     2.857143  0.0  0.0  
52145     2.730159  0.0  0.0  
52146     2.619048  0.0  0.0  
52147     2.539683  0.0  0.0  
52148     2.476190  0.0  0.0  

[52149 rows x 32 columns]

In [3]:
import torch

class FSRDataset(torch.utils.data.Dataset):
    def __init__(self, X_df, y_df, index):
        assert(len(X_df) == len(y_df))
        self.X_df = X_df
        self.y_df = y_df
        self.index = index

    def __len__(self):
        return len(self.index)
    
    def __getitem__(self, idx):
        import numpy as np
        X = self.X_df.loc[self.index[idx]].to_numpy().astype(np.float32)
        y = self.y_df.loc[self.index[idx]].to_numpy().astype(np.float32)
        return X, y

In [4]:
def get_index_splited_by_time(test_size=None):
    from sklearn.model_selection import train_test_split
    train_indexes = []
    test_indexes = []
    for _, group in data.groupby('id'):
        train_index, test_index = train_test_split(group.index, test_size=0.2, shuffle=False)
        train_indexes.append(train_index)
        test_indexes.append(test_index)
    return train_indexes, test_indexes

In [5]:
class LSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layer, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.output_size = output_size
        self.encoder = torch.nn.LSTM(input_size, hidden_size, num_layer)
        self.decoder = torch.nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x, _ = self.encoder(x)
        x = self.decoder(x)
        return x

In [6]:
import torch
from ray.air import session
from ray.air.config import ScalingConfig
from sklearn.metrics import mean_absolute_error, mean_squared_error
import ray.train.torch
import numpy as np
import random
import model as net
import torch.utils.data

def train_loop_per_worker(config):
    ray.train.torch.enable_reproducibility()
    model_name = config['model_name']
    model_args = config['model_args']
    num_epochs = config['num_epochs']
    criterion_name = config['criterion_name']
    optimizer_name = config['optimizer_name']
    lr = config['lr']

    model = getattr(net, model_name)(**model_args)
    model = ray.train.torch.prepare_model(model)
    criterion = getattr(torch.nn, criterion_name)()
    optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)

    train_index, test_index = get_index_splited_by_time()
    train_dataset = FSRDataset(data['FSR_for_force'], data['force'], train_index)
    test_dataset = FSRDataset(data['FSR_for_force'], data['force'], test_index)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=None)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=None)
    train_loader = ray.train.torch.prepare_data_loader(train_loader)
    test_loader = ray.train.torch.prepare_data_loader(test_loader)

    best_rmse = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        criterion.train()
        for X, y in train_loader:
            pred = model(X)
            loss = criterion(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        criterion.eval()
        with torch.no_grad():
            mae = []
            mse = []
            num = []
            for X, y in test_loader:
                pred = model(X)
                mae.append(mean_absolute_error(y, pred.cpu().detach()))
                mse.append(mean_squared_error(y, pred.cpu().detach()))
                num.append(len(y))
            mae = np.array(mae)
            mse = np.array(mse)
            num = np.array(num)
            mae = (mae * num).sum() / sum(num)
            mse = (mse * num).sum() / sum(num)
            
            rmse = mse ** 0.5
            if rmse < best_rmse:
                best_rmse = rmse

            session.report({'MAE': mae, 'RMSE': rmse})
                
    return best_rmse

trainer = ray.train.torch.TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config={
        'batch_size':128,
        'lr':0.0001,
        'model_name':'LSTM',
        'model_args':{
            'input_size':6, 
            'hidden_size':128, 
            'num_layer':4, 
            'output_size':6,
        },
        'num_epochs': 32,
        'criterion_name': 'MSELoss',
        'optimizer_name': 'Adam',
    },
    scaling_config=ScalingConfig(
        num_workers=3,
        use_gpu=False,
        trainer_resources={'CPU':0}
    ),
)

In [7]:
trainer.fit()

2023-06-28 10:26:22,595	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-06-28 10:26:23,797	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Trainer(...)`.


(TorchTrainer pid=84502) 2023-06-28 10:26:29,797	INFO backend_executor.py:137 -- Starting distributed worker processes: ['84566 (172.26.215.93)', '84567 (172.26.215.93)', '84568 (172.26.215.93)']
(RayTrainWorker pid=84566) 2023-06-28 10:26:31,419	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=3]
(RayTrainWorker pid=84566) 2023-06-28 10:26:33,295	INFO train_loop_utils.py:286 -- Moving model to device: cpu
(RayTrainWorker pid=84566) 2023-06-28 10:26:33,296	INFO train_loop_utils.py:346 -- Wrapping provided model in DistributedDataParallel.


Trial name,MAE,RMSE,date,done,experiment_tag,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
TorchTrainer_cb57c_00000,227.397,525.853,2023-06-28_10-28-08,True,0,DESKTOP-0P789CI,32,172.26.215.93,84502,100.355,2.81067,100.355,1687915688,32,cb57c_00000


2023-06-28 10:28:10,209	INFO tune.py:1111 -- Total run time: 106.41 seconds (106.36 seconds for the tuning loop).


Result(
  metrics={'MAE': 227.39678746011765, 'RMSE': 525.8533704863545, 'done': True, 'trial_id': 'cb57c_00000', 'experiment_tag': '0'},
  path='/home/seokj/ray_results/TorchTrainer_2023-06-28_10-26-20/TorchTrainer_cb57c_00000_0_2023-06-28_10-26-23',
  checkpoint=None
)